In [1]:
import os


In [2]:
%pwd

'c:\\Users\\Lenovo\\Documents\\Chicken_DiseseClassification\\research'

In [3]:
os.chdir("C:/Users/Lenovo/Documents/Chicken_DiseseClassification/src")

In [4]:
%pwd

'C:\\Users\\Lenovo\\Documents\\Chicken_DiseseClassification\\src'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        
        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from CNNClassifier.utils.common import get_size
from CNNClassifier import logger



In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"File downloaded: {filename} with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            
        logger.info(f"File extracted at : {unzip_path}")

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    
    print(data_ingestion_config)
    
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e    

[2025-09-28 18:04:58,045 : INFO : common : yaml file: ..\config\config.yaml loaded successfully]
[2025-09-28 18:04:58,047 : INFO : common : yaml file: ..\params.yaml loaded successfully]
[2025-09-28 18:04:58,050 : INFO : common : Created directory at: artifacts]
[2025-09-28 18:04:58,052 : INFO : common : Created directory at: artifacts/data_ingestion]
DataIngestionConfig(root_dir=WindowsPath('artifacts/data_ingestion'), source_URL='https://github.com/nisha132/Datasets/raw/refs/heads/main/Chicken-fecal-images.zip', local_data_file=WindowsPath('artifacts/data_ingestion/data.zip'), unzip_dir=WindowsPath('artifacts/data_ingestion'))
[2025-09-28 18:04:58,054 : INFO : 4131640908 : File already exists of size: 11345 KB]
[2025-09-28 18:04:58,552 : INFO : 4131640908 : File extracted at : artifacts\data_ingestion]
